# Raw operator FP check
Created 05/08/2024

Objectives:
* Check if the raw bounary operators have a definite FP charge.

# Package imports

In [1]:
import sys

In [2]:
sys.path.append("../../")

In [3]:
from itertools import chain, groupby, combinations
import re

from collections import Counter, namedtuple, defaultdict

In [4]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os
import pickle

In [5]:
import numpy as np
from jax import numpy as jnp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import scipy

In [6]:
import quimb as qu
import quimb.tensor as qtn
from quimb.tensor.optimize import TNOptimizer

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:36: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:73: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


# Load data

## Wavefunctions

In [7]:
DATA_DIR_1 = r"../../data/interpolated_trivial_to_nontrivial_fermionic_trivial_proj_rep_200_site_dmrg/"
DATA_DIR_2 = r"../../data/interpolated_nontrivial_fermionic_proj_rep_to_nontrivial_proj_rep_200_site_dmrg/"

In [8]:
def parse_file_name(file_name):
    interpolation = int(file_name.split('_')[0])/100

    return interpolation

In [9]:
loaded_data_triv_proj_rep = dict()
energies_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_1))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_1, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_triv_proj_rep[data_info]=data['wavefunction']
        energies_triv_proj_rep[data_info]=data['energy']

In [10]:
loaded_data_non_triv_proj_rep = dict()
energies_non_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_2))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_2, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_non_triv_proj_rep[data_info]=data['wavefunction']
        energies_non_triv_proj_rep[data_info]=data['energy']

In [11]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for psi in loaded_data_triv_proj_rep.values()
    for i in range(psi.L)
)

Counter({('vL', 'p', 'vR'): 4200})

In [12]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for psi in loaded_data_non_triv_proj_rep.values()
    for i in range(psi.L)
)

Counter({('vL', 'p', 'vR'): 4200})

## Boundary operator solutions

In [13]:
SOL_DIR = r"solutions/"

In [14]:
file_name_pattern = re.compile(r'^(?:non_)?triv_\d\.\d+_\d_\d_\d+\.pickle$')

In [15]:
def parse_file_name(file_name):
    if not bool(file_name_pattern.match(file_name)):
        print(file_name)
        return None

    file_name = '.'.join((file_name.split('.'))[:-1])

    if file_name[0] == 'n':
        proj_rep=1
        b, bs, fs, i = file_name.split('_')[2:]
    elif file_name[0] == 't':
        proj_rep=0
        b, bs, fs, i = file_name.split('_')[1:]
    else:
        return None

    b = float(b)
    bs = int(bs)
    fs = int(fs)
    i = int(i)
    
    return (proj_rep, b, bs, fs, i)

In [16]:
boundary_operator_solutions = dict()

for local_file_name in list(os.walk(SOL_DIR))[0][2]:
    f_name = r"{}/{}".format(SOL_DIR, local_file_name, ignore_unknown=False)

    key = parse_file_name(local_file_name)

    if key is not None:
        with open(f_name, 'rb') as f:
            out = pickle.load(f)
    
            boundary_operator_solutions[key] = out

.DS_Store


### Check values

In [17]:
scores = [float(v[0]._value) for v in boundary_operator_solutions.values()]

In [18]:
pd.Series(scores).describe()

count    573.000000
mean      14.665053
std       22.783958
min        0.001833
25%        0.065218
50%        1.537037
75%       45.585068
max      216.741516
dtype: float64

In [19]:
score_pairs = defaultdict(list)

for k, v in boundary_operator_solutions.items():
    score_pairs[k[:-1]].append((k[-1], float(v[0]._value)))

In [20]:
best_score_pairs = {
    k: min(v, key=lambda x: x[1]) for k, v in score_pairs.items()
}

In [21]:
best_scores = [v[1] for v in best_score_pairs.values()]

In [22]:
len(best_score_pairs)

252

In [23]:
pd.Series(best_scores).describe()

count    252.000000
mean       1.737905
std        8.759257
min        0.001833
25%        0.018629
50%        0.053990
75%        0.203661
max       50.003654
dtype: float64

In [24]:
[k for k, v in best_score_pairs.items() if v[1]>1]

[(0, 0.5, 3, 1),
 (0, 0.5, 3, 0),
 (1, 0.5, 1, 1),
 (1, 0.5, 1, 0),
 (1, 0.5, 3, 0),
 (1, 0.5, 3, 1),
 (0, 0.5, 1, 0),
 (0, 0.5, 1, 1),
 (1, 0.5, 2, 0),
 (1, 0.5, 2, 1)]

In [25]:
best_score_pairs[(0, 0, 1, 0)]

(1, 0.017019841820001602)

In [26]:
best_boundary_operators = {
    k: boundary_operator_solutions[(*k, v[0])]
    for k, v in best_score_pairs.items()
    #if k[1] != 0.5
}

In [27]:
len(best_boundary_operators)

252

# Definitions

In [28]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [29]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [30]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [31]:
fermionic_np_symmetries = [np_I, np_JW]

In [32]:
def generate_problem_rdm(quimb_psi, symmetry_site_pairs, leftmost_symmetry_site,
                         num_symmetry_sites, num_boundary_sites):
    q_top = quimb_psi.copy(deep=True)
    for i, s in symmetry_site_pairs:
        q_top.gate(
            s,
            where=i,
            contract=False,
            inplace=True
        )

    
    indices_to_map = list(chain(
        range(leftmost_symmetry_site-num_boundary_sites, leftmost_symmetry_site),
        range(leftmost_symmetry_site+num_symmetry_sites, leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites)
    ))

    index_mapping = {f'k{i}': f'b{i}' for i in indices_to_map}

    q_bottom = (
        quimb_psi
        .copy()
        .reindex(index_mapping, inplace=True)
        .conj()
    )

    sites_to_contract = {
        'left': list(range(leftmost_symmetry_site-num_boundary_sites)),
        'middle': list(range(leftmost_symmetry_site, leftmost_symmetry_site+num_symmetry_sites)),
        'right': list(range(leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites, quimb_psi.L))
    }

    tags_to_contract = {
        k: [f'I{i}' for i in v]
        for k, v in sites_to_contract.items()
    }

    tn = (q_top & q_bottom)

    tnc = (
        tn
        .contract(tags_to_contract['left'])
        .contract(tags_to_contract['middle'])
        .contract(tags_to_contract['right'])
    )

    return tnc

In [33]:
def generate_rdm_from_tenpy_psi(mps_psi, symmetry_site_pairs,
    leftmost_symmetry_site, num_symmetry_sites, num_boundary_sites):
    
    psi_arrays = list()
    psi_arrays.append(mps_psi.get_B(0, 'Th')[0, ...].to_ndarray())
    for i in range(1, mps_psi.L-1):
        psi_arrays.append(mps_psi.get_B(i).to_ndarray())
    psi_arrays.append(mps_psi.get_B(mps_psi.L-1)[..., 0].to_ndarray())
    
    q1 = (
        qtn
        .tensor_1d
        .MatrixProductState(psi_arrays, shape='lpr')
    )
    
    problem_rdm = generate_problem_rdm(
        q1,
        symmetry_site_pairs,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    return problem_rdm

## Optimisation functions

In [34]:
def split_mpo_pair(mpo_pair):
    ml = qtn.TensorNetwork(
        list(map(mpo_pair.tensor_map.__getitem__, mpo_pair.tag_map['left_mpo']))
    )

    mr = qtn.TensorNetwork(
        list(map(mpo_pair.tensor_map.__getitem__, mpo_pair.tag_map['right_mpo']))
    )

    return (ml, mr)

In [35]:
def overlap_loss_function(ml, mr, rdm_tn, epsilon=0):
    c = (rdm_tn & ml & mr) ^ ...

    c_abs_squared = (
        c
        *jnp.conjugate(c)
    )
    #c_abs_squared = c_abs_squared.astype('float32')
    c_abs = (jnp.sqrt(c_abs_squared+epsilon))

    target = jnp.sqrt(1+epsilon)
    loss = (c_abs - target)**2

    return loss

In [36]:
def overlap_loss_function_mpo_pair(mpo_pair, rdm_tn):
    ml, mr = split_mpo_pair(mpo_pair)

    return overlap_loss_function(ml, mr, rdm_tn)

In [37]:
regex_s = r"^I\d+$"
regex_p = re.compile(regex_s)

In [38]:
def relabel_mpo(mpo, k_label, b_label):
    site_locs = [
        int(k[1:]) for k in mpo.tag_map
        if bool(re.search(regex_p, k))
    ]

    k_in_indices = [f'k{i}' for i in site_locs]
    j_in_indices = [f'b{i}' for i in site_locs]

    k_out_indices = [f'{k_label}{i}' for i in site_locs]
    j_out_indices = [f'{b_label}{i}' for i in site_locs]

    mapping = dict(
        chain(
            zip(k_in_indices, k_out_indices),
            zip(j_in_indices, j_out_indices)
        )
    )

    mpo.reindex(mapping, inplace=True)

In [39]:
def unitarity_tn(tn, total_physical_dim):
    ms = [tn.copy(), tn.copy(), tn.copy()]

    relabel_mpo(ms[0], 'k', 'l')
    relabel_mpo(ms[1], 'm', 'l')
    relabel_mpo(ms[2], 'm', 'b')

    ms[0] = ms[0].conj()
    ms[2] = ms[2].conj()

    n2tn = (tn & tn.conj())
    n2 = n2tn.contract(n2tn.tag_map)
    n4tn = (tn & ms[0] & ms[1] & ms[2])
    n4 = n4tn.contract(n4tn.tag_map)

    out = jnp.real(n4 - 2*n2 + total_physical_dim)

    return out

In [40]:
def overall_loss_function(mpo_pair, rdm_tn, total_physical_dimension,
    unitary_cost_coefficient=1, overlap_cost_coefficient=1, losses=None):
    ml, mr = split_mpo_pair(mpo_pair)

    o_loss = overlap_loss_function(ml, mr, rdm_tn)
    ul_loss = unitarity_tn(ml, total_physical_dimension)
    ur_loss = unitarity_tn(mr, total_physical_dimension)

    out = (
        unitary_cost_coefficient*(ul_loss+ur_loss)
        + overlap_cost_coefficient*o_loss
    )

    out = jnp.real(out)

    if losses is not None:
        losses.append((o_loss, ul_loss, ur_loss))
    return out

## Extract SPT phase functions

In [41]:
def get_right_fp_overlap(rdm, mpo_l, mpo_r, leftmost_symmetry_site,
                     num_symmetry_sites, num_boundary_sites):

    tn = rdm.copy(deep=True)

    fermionic_site_indices = [
        i
        for i in range(
            leftmost_symmetry_site+num_symmetry_sites,
            leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites
        )
        if i % 2 == 1
    ]

    top_fermionic_tensors = [
        (f'k{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'k{i}'])))
        for i in fermionic_site_indices
    ]

    bottom_fermionic_tensors = [
        (f'b{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'b{i}'])))
        for i in fermionic_site_indices
    ]

    for ind, t in top_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            inplace=True
        )

    for ind, t in bottom_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            transposed=True,
            inplace=True
        )

    out = (tn & mpo_l & mpo_r) ^ ...

    return out

In [42]:
def get_left_fp_overlap(rdm, mpo_l, mpo_r, leftmost_symmetry_site,
                     num_symmetry_sites, num_boundary_sites):

    tn = rdm.copy(deep=True)

    fermionic_site_indices = [
        i
        for i in range(
            leftmost_symmetry_site-num_boundary_sites,
            leftmost_symmetry_site
        )
        if i % 2 == 1
    ]

    top_fermionic_tensors = [
        (f'k{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'k{i}'])))
        for i in fermionic_site_indices
    ]

    bottom_fermionic_tensors = [
        (f'b{i}', next(map(tn.tensor_map.__getitem__, tn.ind_map[f'b{i}'])))
        for i in fermionic_site_indices
    ]

    for ind, t in top_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            inplace=True
        )

    for ind, t in bottom_fermionic_tensors:
        t.gate(
            np_JW,
            ind=ind,
            transposed=True,
            inplace=True
        )

    out = (tn & mpo_l & mpo_r) ^ ...

    return out

In [43]:
def get_fp_charges(rdm, ml, mr, left_most_symmetry_site, num_symmetry_sites,
               num_boundary_sites):
    base_overlap = (rdm & ml & mr) ^ ...

    right_fp_overlap = get_right_fp_overlap(
        rdm,
        ml,
        mr,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    left_fp_overlap = get_left_fp_overlap(
        rdm,
        ml,
        mr,
        left_most_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    right_fp_charge = right_fp_overlap/base_overlap
    left_fp_charge = left_fp_overlap/base_overlap

    return (left_fp_charge, right_fp_charge, base_overlap)

## Raw fermionic SPT boundary operator functions

In [72]:
def get_right_raw_fp_charge(mpo_l, mpo_r, leftmost_symmetry_site,
                     num_symmetry_sites, num_boundary_sites):

    tn = mpo_r.copy(deep=True)

    fermionic_site_indices = [
        i
        for i in range(
            leftmost_symmetry_site+num_symmetry_sites,
            leftmost_symmetry_site+num_symmetry_sites+num_boundary_sites
        )
        if i % 2 == 1
    ]

    fermionic_tensors = [
        (
            f'k{i}',
            f'b{i}',
            next(map(tn.tensor_map.__getitem__, tn.ind_map[f'k{i}']))
        )
        for i in fermionic_site_indices
    ]

    for k_ind, b_ind, t in fermionic_tensors:
        t.gate(
            np_JW,
            ind=k_ind,
            inplace=True
        )

        t.gate(
            np_JW,
            ind=b_ind,
            transposed=True,
            inplace=True
        )

    mpo_r_conj = mpo_r.copy(deep=True)
    relabel_mpo(mpo_r_conj, 'b', 'k')
    mpo_r_conj = mpo_r_conj.conj()
    
    contraction = (tn & mpo_r_conj) ^ ...

    return contraction

In [77]:
def get_left_raw_fp_charge(mpo_l, mpo_r, leftmost_symmetry_site,
                     num_symmetry_sites, num_boundary_sites):

    tn = mpo_l.copy(deep=True)

    fermionic_site_indices = [
        i
        for i in range(
            leftmost_symmetry_site-num_boundary_sites,
            leftmost_symmetry_site
        )
        if i % 2 == 1
    ]

    fermionic_tensors = [
        (
            f'k{i}',
            f'b{i}',
            next(map(tn.tensor_map.__getitem__, tn.ind_map[f'k{i}']))
        )
        for i in fermionic_site_indices
    ]

    for k_ind, b_ind, t in fermionic_tensors:
        t.gate(
            np_JW,
            ind=k_ind,
            inplace=True
        )

        t.gate(
            np_JW,
            ind=b_ind,
            transposed=True,
            inplace=True
        )

    mpo_l_conj = mpo_l.copy(deep=True)
    relabel_mpo(mpo_l_conj, 'b', 'k')
    mpo_l_conj = mpo_l_conj.conj()
    
    contraction = (tn & mpo_l_conj) ^ ...

    return contraction

# Extract SPT phase
## Fermionic group cohomology

In [55]:
num_boundary_sites=6
left_most_symmetry_site=60
leftmost_symmetry_site = left_most_symmetry_site
num_symmetry_sites=80
bond_dimension=6

total_physical_dim = 2**9

In [47]:
mpo_pair = best_boundary_operators[0, 0, 1, 0][1]

In [48]:
mpo_pair

TensorNetwork1D(tensors=12, indices=34, L=6, max_bond=6)

In [49]:
ml, mr = split_mpo_pair(mpo_pair)

In [50]:
ml

TensorNetwork(tensors=6, indices=17)

In [56]:
fermionic_site_indices = [
    i
    for i in range(
        leftmost_symmetry_site-num_boundary_sites,
        leftmost_symmetry_site
    )
    if i % 2 == 1
]

In [57]:
top_fermionic_tensors = [
    (f'k{i}', next(map(ml.tensor_map.__getitem__, ml.ind_map[f'k{i}'])))
    for i in fermionic_site_indices
]

In [62]:
help(top_fermionic_tensors[0][1].gate)

Help on method gate in module quimb.tensor.tensor_core:

gate(G, ind, preserve_inds=True, transposed=False, inplace=False) method of quimb.tensor.tensor_core.Tensor instance
    Gate this tensor - contract a matrix into one of its indices without
    changing its indices. Unlike ``contract``, ``G`` is a raw array and the
    tensor remains with the same set of indices. This is like applying:
    
    .. math::
    
        x \leftarrow G x
    
    or if ``transposed=True``:
    
    .. math::
    
        x \leftarrow x G
    
    Parameters
    ----------
    G : 2D array_like
        The matrix to gate the tensor index with.
    ind : str
        Which index to apply the gate to.
    preserve_inds : bool, optional
        If ``True``, the order of the indices is preserved, otherwise the
        gated index will be left at the first axis, avoiding a transpose.
    transposed : bool, optional
        If ``True``, the gate is effectively transpose and applied, or
        equivalently, 

In [78]:
fp_charges = dict()

for k, ops in best_boundary_operators.items():
    ml, mr = split_mpo_pair(ops[1])

    right_raw_fp_charge = get_right_raw_fp_charge(
        ml,
        mr,
        leftmost_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    left_raw_fp_charge = get_left_raw_fp_charge(
        ml,
        mr,
        leftmost_symmetry_site,
        num_symmetry_sites,
        num_boundary_sites
    )

    fp_charges[k] = (left_raw_fp_charge, right_raw_fp_charge)

In [79]:
fp_charges

{(0, 0.9, 3, 1): ((33.64170491405497+2.3374691500155365e-06j),
  (5.9946966165095015-1.2975242371648932e-06j)),
 (1, 0.35, 2, 1): ((0.6464549678562967-3.4019638306403976e-06j),
  (0.1549383613543398+1.2870554602528728e-05j)),
 (0, 0.9, 3, 0): ((65.06008359739417+3.324945783145239e-07j),
  (45.15460252906394-4.772930339846937e-08j)),
 (1, 0.35, 2, 0): ((-2.7595395166508663-7.640247510209974e-07j),
  (2.475776842415754+2.545915139595678e-06j)),
 (0, 0.45, 1, 1): ((242.50252014073567-2.845093639791685e-06j),
  (129.82929045670335+1.081471788566546e-06j)),
 (1, 0.25, 1, 1): ((0.3336313556944681-8.122313147396198e-06j),
  (11.570799996497374-3.2180724076624756e-06j)),
 (1, 0.25, 1, 0): ((1.363922649738596+1.2014863562725964e-05j),
  (2.32891976789395+1.0907775136459463e-05j)),
 (1, 0.4, 3, 1): ((36.226184910006474+3.153487696749835e-06j),
  (6.102517277234844+1.6285658128722957e-06j)),
 (1, 0.05, 2, 1): ((-0.173216891099905-3.025558502400827e-06j),
  (-0.07622750885834861-1.0353041579591604

# Conclusion
Boundary operators do not have a definite fermion parity.